In [409]:
!pip install praw
import praw
import pandas as pd
import numpy as np
import datetime as dt
from datetime import date, time
import warnings
from bs4 import BeautifulSoup
import requests
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.linear_model import LassoCV, RidgeCV
from sklearn import metrics
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from statsmodels.stats.outliers_influence import variance_inflation_factor
from matplotlib import pyplot as plt
warnings.filterwarnings('ignore')

## Connect to account & scrape from reddit

In [2]:
r = praw.Reddit(
    client_id="XXX",
    client_secret="XXX",
    refresh_token="XXXX",
    user_agent="XXX",
)
print(r.user.me())

jreji


In [3]:
sub = r.subreddit('AmItheAsshole')
top_subreddit = sub.top(limit=500)

topics_dict = { "title":[],
                "score":[],
                "id":[], "url":[], 
                "comms_num": [],
                "created": [], 
                "body":[]}

In [4]:
for sub in top_subreddit:
    topics_dict["title"].append(sub.title)
    topics_dict["score"].append(sub.score)
    topics_dict["id"].append(sub.id)
    topics_dict["url"].append(sub.url)
    topics_dict["comms_num"].append(sub.num_comments)
    topics_dict["created"].append(sub.created)
    topics_dict["body"].append(sub.selftext)

In [5]:
topics_data = pd.DataFrame(topics_dict)
topics_data.to_csv("RedditData.csv")

In [6]:
def get_date(created):
    return dt.datetime.fromtimestamp(created)

## Convert to a dataframe and fix timestamp

In [7]:
_timestamp = topics_data["created"].apply(get_date)
topics_data = topics_data.assign(timestamp = _timestamp)
topics_data

,title,score,id,url,comms_num,created,body,timestamp
0,AITA for telling my wife the lock on my daught...,81021,ocx94s,https://www.reddit.com/r/AmItheAsshole/comment...,5296,1.625316e+09,My brother in-law (Sammy) lost his home shortl...,2021-07-03 08:36:22
1,META: This sub is moving towards a value syste...,80920,d6xoro,https://www.reddit.com/r/AmItheAsshole/comment...,6209,1.568998e+09,I’ve enjoyed reading and posting on this sub f...,2019-09-20 12:51:40
2,"UPDATE, AITA for despising my mentally handica...",72780,azvko1,https://www.reddit.com/r/AmItheAsshole/comment...,1988,1.552322e+09,"I'm back like I said I would be,. My [original...",2019-03-11 12:41:02
3,AITA For suing my girlfriend after she had my ...,70813,gr8bp3,https://www.reddit.com/r/AmItheAsshole/comment...,2769,1.590536e+09,I'll try to keep this short. I had a [1967 Imp...,2020-05-26 19:35:45
4,UPDATE: AITA for wanting to go to the funeral ...,67574,cjetsa,https://www.reddit.com/r/AmItheAsshole/comment...,2,1.564423e+09,I want to sincerely thank everyone who comment...,2019-07-29 13:58:12
...,...,...,...,...,...,...,...,...
495,AITA for not wanting to celebrate my mom on MY...,25396,suo7nj,https://www.reddit.com/r/AmItheAsshole/comment...,3912,1.645103e+09,I have three siblings between the ages of 10 a...,2022-02-17 07:57:54
496,AITA for posting a tribute of my late-girlfrie...,25370,d2uish,https://www.reddit.com/r/AmItheAsshole/comment...,3352,1.568226e+09,I lost my late-girlfriend on 9/11 when TWC tow...,2019-09-11 14:20:46
497,AITA for being upset and leaving when my siste...,25351,ivrh66,https://www.reddit.com/r/AmItheAsshole/comment...,2762,1.600518e+09,[UPDATE ONE](https://www.reddit.com/r/JUSTNOFA...,2020-09-19 08:14:43
498,AITA for calling a woman an idiot when she ref...,25334,lj2pxa,https://www.reddit.com/r/AmItheAsshole/comment...,1701,1.613231e+09,My (20f) 2yo daughter has had a heart transpla...,2021-02-13 10:38:19


## Bastime calculation functions

In [8]:
# Creating the basetime metric - 72 hours after post is published
# Assuming 28 days in each month
def get_basetime(timestamp):
    if (timestamp.day >= 28):
        baseDay = 3
        if (timestamp.month == 12):
            baseMonth = 1
            baseYear = timestamp.year+1
        else:
            baseMonth = timestamp.month+1
            baseYear = timestamp.year
    elif (timestamp.day == 27):
        baseDay = 2
        if (timestamp.month == 12):
            baseMonth = 1
            baseYear = timestamp.year+1
        else:
            baseMonth = timestamp.month+1
            baseYear = timestamp.year
    elif (timestamp.day == 26):
        baseDay = 1
        if (timestamp.month == 12):
            baseMonth = 1
            baseYear = timestamp.year+1
        else:
            baseMonth = timestamp.month+1
            baseYear = timestamp.year
    else:
        baseDay = timestamp.day + 3
        baseMonth = timestamp.month
        baseYear = timestamp.year
        
    return timestamp.combine(date(baseYear, baseMonth, baseDay), 
                                 time(timestamp.hour, timestamp.minute, timestamp.second))

# Creating timestamps 24 hours before relative to basetime
# Assuming 28 days in each month
def get24BeforeBasetime(timestamp):
    if (timestamp.day == 1):
        prevDay = 28
        if (timestamp.month == 1):
            prevMonth = 12
            prevYear = timestamp.year-1
        else:
            prevMonth = timestamp.month-1
            prevYear = timestamp.year 
    else:
        prevDay = timestamp.day-1
        prevMonth = timestamp.month
        prevYear = timestamp.year
        
    return timestamp.combine(date(prevYear, prevMonth, prevDay), 
                             time(timestamp.hour, timestamp.minute, timestamp.second))

# Creating timestamps 24 hours after relative to basetime
# Assuming 28 days in each month
def get24After(timestamp):
    if (timestamp.day >= 28):
        nextDay = 1
        if (timestamp.month == 12):
            nextMonth = 1
            nextYear = timestamp.year+1
        else:
            nextMonth = timestamp.month+1
            nextYear = timestamp.year        
    else:
        nextDay = timestamp.day+1
        nextMonth = timestamp.month
        nextYear = timestamp.year
        
    return timestamp.combine(date(nextYear, nextMonth, nextDay), 
                             time(timestamp.hour, timestamp.minute, timestamp.second))

# Creating timestamps 48 hours before relative to basetime
# Assuming 28 days in each month
def get48BeforeBasetime(timestamp):
    if (timestamp.day == 1):
        twoPrevDay = 27
        if (timestamp.month == 1):
            twoPrevMonth = 12
            twoPrevyear = timestamp.year-1
        else:
            twoPrevMonth = timestamp.month-1
            twoPrevyear = timestamp.year 
    elif (timestamp.day == 2):
        twoPrevDay = 28
        if (timestamp.month == 1):
            twoPrevMonth = 12
            twoPrevyear = timestamp.year-1
        else:
            twoPrevMonth = timestamp.month-1
            twoPrevyear = timestamp.year 
    else:
        twoPrevDay = timestamp.day-2
        twoPrevMonth = timestamp.month
        twoPrevyear = timestamp.year


    return timestamp.combine(date(twoPrevyear, twoPrevMonth, twoPrevDay), 
                                    time(timestamp.hour, timestamp.minute, timestamp.second))

In [9]:
def get_CommentsSource(timestamp, df):
    comments_Before = []

    basetime = get_basetime(timestamp)
    prevDayT = get24BeforeBasetime(basetime)
    twoprevDayT = get48BeforeBasetime(basetime) 
    nextDayT = get24After(timestamp)

    # Calculate metrics for total comments before
    mean = df[(df['timestamp'] < basetime)]['comms_num'].mean()
    std = df[(df['timestamp'] < basetime)]['comms_num'].std()
    min_ = df[(df['timestamp'] < basetime)]['comms_num'].min()
    max_ = df[(df['timestamp'] < basetime)]['comms_num'].max()
    median = df[(df['timestamp'] < basetime)]['comms_num'].median()
    
    # Calculate metrics for total comments 24 hours before
    mean_24 = df[(df['timestamp'] < basetime) & (df['timestamp'] > prevDayT)]['comms_num'].mean()
    std_24 = df[(df['timestamp'] < basetime) & (df['timestamp'] > prevDayT)]['comms_num'].std()
    min_24 = df[(df['timestamp'] < basetime) & (df['timestamp'] > prevDayT)]['comms_num'].min()
    max_24 = df[(df['timestamp'] < basetime) & (df['timestamp'] > prevDayT)]['comms_num'].max()
    median_24 = df[(df['timestamp'] < basetime) & (df['timestamp'] > prevDayT)]['comms_num'].median()
    
    # Calculate metrics for total comments between 48 & 24 hours
    mean_int = df[(df['timestamp'] < prevDayT) & (df['timestamp'] > twoprevDayT)]['comms_num'].mean()
    std_int = df[(df['timestamp'] < prevDayT) & (df['timestamp'] > twoprevDayT)]['comms_num'].std()
    min_int = df[(df['timestamp'] < prevDayT) & (df['timestamp'] > twoprevDayT)]['comms_num'].min()
    max_int = df[(df['timestamp'] < prevDayT) & (df['timestamp'] > twoprevDayT)]['comms_num'].max()
    median_int = df[(df['timestamp'] < prevDayT) & (df['timestamp'] > twoprevDayT)]['comms_num'].median()
    
    # Calculate metrics for total comments in the first 24 hours
    mean_24After = df[(df['timestamp'] < nextDayT) & (df['timestamp'] > timestamp)]['comms_num'].mean()
    std_24After = df[(df['timestamp'] < nextDayT) & (df['timestamp'] > timestamp)]['comms_num'].std()
    min_24After = df[(df['timestamp'] < nextDayT) & (df['timestamp'] > timestamp)]['comms_num'].min()
    max_24After = df[(df['timestamp'] < nextDayT) & (df['timestamp'] > timestamp)]['comms_num'].max()
    median_24After = df[(df['timestamp'] < nextDayT) & (df['timestamp'] > timestamp)]['comms_num'].median()
    
    # The differnce between 52 & 53
    dif = (df[(df['timestamp'] < basetime) & (df['timestamp'] > prevDayT)]['comms_num'].sum() - 
                df[(df['timestamp'] < prevDayT) & (df['timestamp'] > twoprevDayT)]['comms_num'].sum())
    int_mean = dif.mean()
    int_std = dif.std()
    int_min = dif.min()
    int_max = dif.max()
    int_median = dif

    # Append to list
    comments_Before.append(mean)
    comments_Before.append(std)
    comments_Before.append(min_)
    comments_Before.append(max_)
    comments_Before.append(median)
    comments_Before.append(mean_24)
    comments_Before.append(std_24)
    comments_Before.append(min_24)
    comments_Before.append(max_24)
    comments_Before.append(median_24)
    comments_Before.append(mean_int)
    comments_Before.append(std_int)
    comments_Before.append(min_int)
    comments_Before.append(max_int)
    comments_Before.append(median_int)
    comments_Before.append(mean_24After)
    comments_Before.append(std_24After)
    comments_Before.append(min_24After)
    comments_Before.append(max_24After)
    comments_Before.append(median_24After)
    comments_Before.append(int_mean)
    comments_Before.append(int_std)
    comments_Before.append(int_min)
    comments_Before.append(int_max)
    comments_Before.append(int_median)

    return comments_Before

## Creating data specific columns

In [146]:
topics_data['mean_totalcommentsBefore_Source'] = 0
topics_data['std_totalcommentsBefore_Source'] = 0
topics_data['min_totalcommentsBefore_Source'] = 0
topics_data['max_totalcommentsBefore_Source'] = 0
topics_data['median_totalcommentsBefore_Source'] = 0

topics_data['mean_comments24Before_Source'] = 0
topics_data['std_comments24Before_Source'] = 0
topics_data['min_comments24Before_Source'] = 0
topics_data['max_comments24Before_Source'] = 0
topics_data['median_comments24Before_Source'] = 0

topics_data['mean_commentsIntervalBefore_Source'] = 0
topics_data['std_commentsIntervalBefore_Source'] = 0
topics_data['min_commentsIntervalBefore_Source'] = 0
topics_data['max_commentsIntervalBefore_Source'] = 0
topics_data['median_commentsIntervalBefore_Source'] = 0

topics_data['mean_comments24After_Source'] = 0
topics_data['std_comments24After_Source'] = 0
topics_data['min_comments24After_Source'] = 0
topics_data['max_comments24After_Source'] = 0
topics_data['median_comments24After_Source'] = 0

topics_data['mean_comments24Interval_Source'] = 0
topics_data['std_comments24Interval_Source'] = 0
topics_data['min_comments24Interval_Source'] = 0
topics_data['max_comments24Interval_Source'] = 0
topics_data['median_comments24Interval_Source'] = 0

for i in range(len(topics_data)):
    temp = get_CommentsSource(topics_data['timestamp'][i], topics_data)
    topics_data['mean_totalcommentsBefore_Source'][i] = temp[0]
    topics_data['std_totalcommentsBefore_Source'][i] = temp[1]
    topics_data['min_totalcommentsBefore_Source'][i] = temp[2]
    topics_data['max_totalcommentsBefore_Source'][i] = temp[3]
    topics_data['median_totalcommentsBefore_Source'][i] = temp[4]
    
    topics_data['mean_comments24Before_Source'][i] = temp[5]
    topics_data['std_comments24Before_Source'][i] = temp[6]
    topics_data['min_comments24Before_Source'][i] = temp[7]
    topics_data['max_comments24Before_Source'][i] = temp[8]
    topics_data['median_comments24Before_Source'][i] = temp[9]
    
    topics_data['mean_commentsIntervalBefore_Source'][i] = temp[10]
    topics_data['std_commentsIntervalBefore_Source'][i] = temp[11]
    topics_data['min_commentsIntervalBefore_Source'][i] = temp[12]
    topics_data['max_commentsIntervalBefore_Source'][i] = temp[13]
    topics_data['median_commentsIntervalBefore_Source'][i] = temp[14]
    
    topics_data['mean_comments24After_Source'][i] = temp[15]
    topics_data['std_comments24After_Source'][i] = temp[16]
    topics_data['min_comments24After_Source'][i] = temp[17]
    topics_data['max_comments24After_Source'][i] = temp[18]
    topics_data['median_comments24After_Source'][i] = temp[19]
    
    topics_data['mean_comments24Interval_Source'][i] = temp[20]
    topics_data['std_comments24Interval_Source'][i] = temp[21]
    topics_data['min_comments24Interval_Source'][i] = temp[22]
    topics_data['max_comments24Interval_Source'][i] = temp[23]
    topics_data['median_comments24Interval_Source'][i] = temp[24]
    #print("Iteration", i)

topics_data

,title,score,id,url,comms_num,created,body,timestamp,mean_totalcommentsBefore_Source,std_totalcommentsBefore_Source,...,std_comments24After_Source,min_comments24After_Source,max_comments24After_Source,median_comments24After_Source,mean_comments24Interval_Source,std_comments24Interval_Source,min_comments24Interval_Source,max_comments24Interval_Source,median_comments24Interval_Source,clean_text
0,AITA for telling my wife the lock on my daught...,81021,ocx94s,https://www.reddit.com/r/AmItheAsshole/comment...,5296,1.625316e+09,My brother in-law (Sammy) lost his home shortl...,2021-07-03 08:36:22,2512,1643.000000,...,NaN,3043.0,3043.0,3043.0,-5653,0,-5653,-5653,-5653,brother law sammy lose home shortly divorce mo...
1,META: This sub is moving towards a value syste...,80920,d6xoro,https://www.reddit.com/r/AmItheAsshole/comment...,6209,1.568998e+09,I’ve enjoyed reading and posting on this sub f...,2019-09-20 12:51:40,3239,2105.000000,...,NaN,NaN,NaN,NaN,2584,0,2584,2584,2584,ive enjoy read post sub many month feel like i...
2,"UPDATE, AITA for despising my mentally handica...",72780,azvko1,https://www.reddit.com/r/AmItheAsshole/comment...,1988,1.552322e+09,"I'm back like I said I would be,. My [original...",2019-03-11 12:41:02,1979,1331.000000,...,NaN,NaN,NaN,NaN,610,0,610,610,610,back like say would original post http www red...
3,AITA For suing my girlfriend after she had my ...,70813,gr8bp3,https://www.reddit.com/r/AmItheAsshole/comment...,2769,1.590536e+09,I'll try to keep this short. I had a [1967 Imp...,2020-05-26 19:35:45,2995,1981.000000,...,NaN,2012.0,2012.0,2012.0,4562,0,4562,4562,4562,try keep short impala door http imgur com hlbp...
4,UPDATE: AITA for wanting to go to the funeral ...,67574,cjetsa,https://www.reddit.com/r/AmItheAsshole/comment...,2,1.564423e+09,I want to sincerely thank everyone who comment...,2019-07-29 13:58:12,2944,2046.000000,...,634.981890,677.0,1575.0,1126.0,0,0,0,0,0,want sincerely thank everyone comment everyone...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,AITA for not wanting to celebrate my mom on MY...,25396,suo7nj,https://www.reddit.com/r/AmItheAsshole/comment...,3912,1.645103e+09,I have three siblings between the ages of 10 a...,2022-02-17 07:57:54,2565,1598.691308,...,NaN,NaN,NaN,NaN,0,0,0,0,0,three sibling age old fourth every year every ...
496,AITA for posting a tribute of my late-girlfrie...,25370,d2uish,https://www.reddit.com/r/AmItheAsshole/comment...,3352,1.568226e+09,I lost my late-girlfriend on 9/11 when TWC tow...,2019-09-11 14:20:46,3211,2062.570110,...,1754.331924,366.0,2847.0,1606.5,0,0,0,0,0,lose late girlfriend twc tower collapse th flo...
497,AITA for being upset and leaving when my siste...,25351,ivrh66,https://www.reddit.com/r/AmItheAsshole/comment...,2762,1.600518e+09,[UPDATE ONE](https://www.reddit.com/r/JUSTNOFA...,2020-09-19 08:14:43,2641,1765.018725,...,NaN,NaN,NaN,NaN,0,0,0,0,0,update one http www reddit com r justnofamily ...
498,AITA for calling a woman an idiot when she ref...,25334,lj2pxa,https://www.reddit.com/r/AmItheAsshole/comment...,1701,1.613231e+09,My (20f) 2yo daughter has had a heart transpla...,2021-02-13 10:38:19,2530,1680.413309,...,NaN,NaN,NaN,NaN,0,0,0,0,0,f yo daughter heart transplant due chd severel...


## Reading in Bow model from Imad's work

In [391]:
features_df = pd.read_excel('reddit_data_with_bow.xlsx')
features_df.drop(columns='Unnamed: 0', inplace=True)
features_df.head()

,title,score,id,url,comms_num,created,body,timestamp,mean_totalcommentsBefore_Source,std_totalcommentsBefore_Source,...,whole,marry,word,spend,mad,basically,instead,question,thanks,probably
0,AITA for telling my wife the lock on my daught...,81020,ocx94s,https://www.reddit.com/r/AmItheAsshole/comment...,5296,1625315782,My brother in-law (Sammy) lost his home shortl...,2021-07-03 12:36:22,2508,1642.0,...,0,0,0,1,1,0,0,0,0,0
1,META: This sub is moving towards a value syste...,80910,d6xoro,https://www.reddit.com/r/AmItheAsshole/comment...,6209,1568998300,I’ve enjoyed reading and posting on this sub f...,2019-09-20 16:51:40,3239,2105.0,...,0,0,0,0,0,0,0,0,1,1
2,"UPDATE, AITA for despising my mentally handica...",72777,azvko1,https://www.reddit.com/r/AmItheAsshole/comment...,1988,1552322462,"I'm back like I said I would be,. My [original...",2019-03-11 16:41:02,1979,1331.0,...,1,0,0,1,1,0,1,0,1,0
3,AITA For suing my girlfriend after she had my ...,70802,gr8bp3,https://www.reddit.com/r/AmItheAsshole/comment...,2769,1590536145,I'll try to keep this short. I had a [1967 Imp...,2020-05-26 23:35:45,2991,1975.0,...,1,0,0,1,0,0,0,1,0,0
4,UPDATE: AITA for wanting to go to the funeral ...,67571,cjetsa,https://www.reddit.com/r/AmItheAsshole/comment...,2,1564423092,I want to sincerely thank everyone who comment...,2019-07-29 17:58:12,2945,2046.0,...,0,0,0,0,0,0,1,0,0,0


## Renaming the bow features and dropping unnecessary ones

In [392]:
count=1
for index in range(35,235):
    #print((features_df.loc[:,features_df.columns[index]]))
    #features_df[index] = str(count)+"_bagOfWords_Post"
    val = features_df.columns[index]
    features_df.rename(columns = {val: str(count)+"_bagOfWords_Post"}, inplace=True)
    count+=1

In [393]:
features_df.drop(columns='clean_text', inplace=True)
features_df.drop(columns='key_word', inplace=True)
features_df.columns

Index(['title', 'score', 'id', 'url', 'comms_num', 'created', 'body',
       'timestamp', 'mean_totalcommentsBefore_Source',
       'std_totalcommentsBefore_Source',
       ...
       '191_bagOfWords_Post', '192_bagOfWords_Post', '193_bagOfWords_Post',
       '194_bagOfWords_Post', '195_bagOfWords_Post', '196_bagOfWords_Post',
       '197_bagOfWords_Post', '198_bagOfWords_Post', '199_bagOfWords_Post',
       '200_bagOfWords_Post'],
      dtype='object', length=233)

## Function to create weekday binary

In [394]:
def getDay(timestamp):
    weekdayBinary = []
    if(timestamp.day_name() == "Monday"):
        weekDayBinay = [1,0,0,0,0,0,0]
    elif (timestamp.day_name() == "Tuesday"):
        weekDayBinay = [0,1,0,0,0,0,0]
    elif (timestamp.day_name() == "Wednesday"):
        weekDayBinay = [0,0,1,0,0,0,0]
    elif (timestamp.day_name() == "Thursday"):
        weekDayBinay = [0,0,0,1,0,0,0]
    elif (timestamp.day_name() == "Friday"):
        weekDayBinay = [0,0,0,0,1,0,0]
    elif (timestamp.day_name() == "Saturday"):
        weekDayBinay = [0,0,0,0,0,1,0]
    elif (timestamp.day_name() == "Sunday"):
        weekDayBinay = [0,0,0,0,0,0,1]
    return weekDayBinay

In [395]:
features_df['monday_Basetime'] = 0
features_df['tuesday_Basetime'] = 0
features_df['wednesday_Basetime'] = 0
features_df['thursday_Basetime'] = 0
features_df['friday_Basetime'] = 0
features_df['saturday_Basetime'] = 0
features_df['sunday_Basetime'] = 0

features_df['monday_Post'] = 0
features_df['tuesday_Post'] = 0
features_df['wednesday_Post'] = 0
features_df['thursday_Post'] = 0
features_df['friday_Post'] = 0
features_df['saturday_Post'] = 0
features_df['sunday_Post'] = 0


for i in range(len(features_df)):
    basetime = get_basetime(features_df['timestamp'][i])
    tempBasetime = getDay(basetime)
    tempPost = getDay(features_df['timestamp'][i])
    
    features_df['monday_Basetime'][i] = tempBasetime[0]
    features_df['tuesday_Basetime'][i] = tempBasetime[1]
    features_df['wednesday_Basetime'][i] = tempBasetime[2]
    features_df['thursday_Basetime'][i] = tempBasetime[3]
    features_df['friday_Basetime'][i] = tempBasetime[4]
    features_df['saturday_Basetime'][i] = tempBasetime[5] 
    features_df['sunday_Basetime'][i] = tempBasetime[6]
    
    features_df['monday_Post'][i] = tempPost[0]
    features_df['tuesday_Post'][i] = tempPost[1]
    features_df['wednesday_Post'][i] = tempPost[2]
    features_df['thursday_Post'][i] = tempPost[3]
    features_df['friday_Post'][i] = tempPost[4]
    features_df['saturday_Post'][i] = tempPost[5] 
    features_df['sunday_Post'][i] = tempPost[6]

In [396]:
features_df["Length_Post"] = features_df["body"].str.len()
features_df.head()

,title,score,id,url,comms_num,created,body,timestamp,mean_totalcommentsBefore_Source,std_totalcommentsBefore_Source,...,saturday_Basetime,sunday_Basetime,monday_Post,tuesday_Post,wednesday_Post,thursday_Post,friday_Post,saturday_Post,sunday_Post,Length_Post
0,AITA for telling my wife the lock on my daught...,81020,ocx94s,https://www.reddit.com/r/AmItheAsshole/comment...,5296,1625315782,My brother in-law (Sammy) lost his home shortl...,2021-07-03 12:36:22,2508,1642.0,...,0,0,0,0,0,0,0,1,0,2664
1,META: This sub is moving towards a value syste...,80910,d6xoro,https://www.reddit.com/r/AmItheAsshole/comment...,6209,1568998300,I’ve enjoyed reading and posting on this sub f...,2019-09-20 16:51:40,3239,2105.0,...,0,0,0,0,0,0,1,0,0,3266
2,"UPDATE, AITA for despising my mentally handica...",72777,azvko1,https://www.reddit.com/r/AmItheAsshole/comment...,1988,1552322462,"I'm back like I said I would be,. My [original...",2019-03-11 16:41:02,1979,1331.0,...,0,0,1,0,0,0,0,0,0,5437
3,AITA For suing my girlfriend after she had my ...,70802,gr8bp3,https://www.reddit.com/r/AmItheAsshole/comment...,2769,1590536145,I'll try to keep this short. I had a [1967 Imp...,2020-05-26 23:35:45,2991,1975.0,...,0,0,0,1,0,0,0,0,0,4554
4,UPDATE: AITA for wanting to go to the funeral ...,67571,cjetsa,https://www.reddit.com/r/AmItheAsshole/comment...,2,1564423092,I want to sincerely thank everyone who comment...,2019-07-29 17:58:12,2945,2046.0,...,1,0,1,0,0,0,0,0,0,1142


In [397]:
features_df['totalComments'] = features_df['comms_num']
features_df.drop(columns= ['title', 'score', 'id', 'url', 'created', 'body', 'timestamp', 'comms_num'], inplace=True)

In [398]:
features_df.head()

,mean_totalcommentsBefore_Source,std_totalcommentsBefore_Source,min_totalcommentsBefore_Source,max_totalcommentsBefore_Source,median_totalcommentsBefore_Source,mean_comments24Before_Source,std_comments24Before_Source,min_comments24Before_Source,max_comments24Before_Source,median_comments24Before_Source,...,sunday_Basetime,monday_Post,tuesday_Post,wednesday_Post,thursday_Post,friday_Post,saturday_Post,sunday_Post,Length_Post,totalComments
0,2508,1642.0,1,11668,2203,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,1,0,2664,5296
1,3239,2105.0,2,8821,2783,2096.0,2725.189535,169.0,4023.0,2096.0,...,0,0,0,0,0,1,0,0,3266,6209
2,1979,1331.0,267,4307,1988,611.0,NaN,611.0,611.0,611.0,...,0,1,0,0,0,0,0,0,5437,1988
3,2991,1975.0,2,11668,2659,1387.5,1039.278436,445.0,2739.0,1183.0,...,0,0,1,0,0,0,0,0,4554,2769
4,2945,2046.0,2,7869,2689,NaN,NaN,NaN,NaN,NaN,...,0,1,0,0,0,0,0,0,1142,2
